## Апісаньне
Вітаю!

Гэты інтэрактыўны ноўтбук дапаможа табе спампаваць аўдыя зь любога відэа на youtube. 
Без усталяваньня лішніх плагінаў у свой браўзер і без аніякай рэкламы.

Мабыць пазьней я дадам магчымасьць спампоўваць і відэа.

Увесь код выконваецца на бясплатнай віртуальнай машыне ў воблаку Google Colab. Таму табе не спатрэбіцца нічога ўсталёўваць на свой камп'ютар. 

### Запуск
* Каб запусьціць ноўтбук табе спатрэбіца толькі 
  **зайсьці ў свой Google-акаўнт**, каб атрымаць доступ да бясплатнай віртуальнай машыны ў воблаку Google Colab
* Ніякія даныя пра твой акаўнт не выкарыстоўваюцца.<br>
  Доступ да твайго Google-дыску таксама не адбываецца.
* Табе неабходна толькі ўвесьці спасылку на відэа ў форму ніжэй і запусьціць **"клеткі"** з кодам
* Запускаць клеткі можна з дапамогай трохкутнічка ў левым верхнім куце клеткі
* Пры першым запуску віртуальнай машыны неабходна выканаць клеткі "Ініцыялізацыі"
* Калі захочаш, то можаш адкрыць "Праваднік" на віртуальнай машыне кнопкай з выявай тэчкі ў самым левым куце старонкі. Але гэта рабіць неабавязкова: ў ноўтбуку ўжо ёсьць гатовы код, які спампуе ўсе файлы зь віртуальнай машыны на твой лакальны камп'ютар
* Па змоўчаньні аўдыяфайлы спампоўваюцца на віртуальную машыну ў тэчку `audio`.


Код выкарыстоўвае бібліятэку [yt-dlp](https://github.com/yt-dlp/yt-dlp). Насамрэч усё можна зрабіць у 2 радкі коду (таму можаш сьмела эксьперыментаваць самастойна!), але для максімальна простага ўзаемадзеяньня з карыстальнікам коду тут трошку больш :)

## Ініцыялізацыя
Ініцыялізуем віртуальную машыну.

Можа заняць пару секунд - пачакай.

In [ ]:
# !python3 -m pip install -U yt-dlp > /dev/null

# currently there is a bug with changed format of youtube channel names:
# https://github.com/yt-dlp/yt-dlp/issues/6247
# hack: 
#  * do not use pre-built python library with outdated code
#  * copy original repo and run using yt-dlp.sh script instead
!if [ ! -d yt-dlp ]; then git clone https://github.com/yt-dlp/yt-dlp.git > /dev/null; fi

import glob
import os
import shutil
import subprocess
from IPython import display as ipd

from google.colab import files

!mkdir -p audio


def convert_audio_to_mp3():
    """ Функцыя для канвертацыі аўдыя ў .mp3 фармат """

    for fp in glob.glob('audio/*'):
        if not fp.endswith('.mp3'):
            new_fp = os.path.splitext(fp)[0] + '.mp3'
            
            if not os.path.isfile(new_fp):    
                print(f'Канвертуем "{fp}" у "{new_fp}"')
                
                try:
                    res = subprocess.run(
                        ['ffmpeg', '-i', fp, '-acodec', 'libmp3lame', new_fp], 
                        capture_output=True, check=True
                    )
                except subprocess.CalledProcessError as e:
                    print('subprocess call ended with non-zero exit code. '
                          f'stderr message:\n{e.stderr.decode()}')
            else:
                print(f'Файл "{new_fp}" ужо спампаваны. '
                    'Калі хочаш спампаваць яго нанава, '
                    'то спачатку выдалі стары файл.'
                )

            os.unlink(fp)  # remove original file

## Спампаваць аўдыя зь youtube на віртуальную машыну
* Пазнач у форме справа спасылку на відэа
* Клетку можна перазапускаць некалькі разоў са спасылкамі на розныя відэа.<br>Усе відэа будуць спампаваныя на віртуальную машыну
* Усе аўдыяфайлы будуць аўтаматычна сканвертаваныя ў `.mp3`.<br>
  Гэтую опцыю можна выключыць - тады аўдыяфайлы будуць захаваныя ў зыходным фармаце (найчасьцей гэта `.opus` фармат)

In [ ]:
youtube_video_link = "https://www.youtube.com/watch?v=ZH_7s5MPWm0" #@param {type:"string"}
convert_to_mp3 = True #@param {type:"boolean"}

assert len(youtube_video_link) > 0, 'Калі ласка, увядзі спасылку на youtube відэа'
print(f'Пачынаецца спампоўка аўдыя з: "{youtube_video_link}"')

## run from cloned repo instead of using built python library.
# !yt-dlp -f bestaudio --extract-audio -o "%(title)s.%(ext)s" "{youtube_video_link}"
!yt-dlp/yt-dlp.sh -f bestaudio --extract-audio -o "audio/%(title)s.%(ext)s"  "{youtube_video_link}"

if convert_to_mp3 is True:
    print()
    convert_audio_to_mp3()

print('\nГатова!')

## Прайграць спампаваныя на віртуальную машыну аўдыя
* Тут ты можаш праверыць спампаваныя на віртуальную машыну аўдыя 

In [ ]:
for fp in glob.glob('audio/*'):
    print(fp)
    display(ipd.Audio(fp))

## Захаваць спампаваныя аўдыя на свой камп'ютар
* Запусьці клетку, і ўсе аўдыяфайлы аўтаматычна спампуюцца на твой камп'ютар

In [ ]:
for fp in glob.glob('audio/*'):
    print(f'Пачынаю спампоўку файла "{fp}" на твой камп\'ютар')
    files.download(fp)

## Ачысьціць тэчку са спампаванымі файламі на воблаку (неабавязкова)
* Калі спатрэбіцца, можаш выдаліць спампаваныя файлы на воблаку. Напрыклад калі захочаш спампаваць пэўны файл нанава.
* Празь нейкі час віртуальная машына выключыцца самастойна, а ўсе даныя будуць выдалены. Таму можаш не выконваць гэтую клетку

In [ ]:
if os.path.isdir('audio'):
    shutil.rmtree('audio')
os.makedirs('audio')
print('Тэчка з аўдыя на віртуальная машыне ачышчаная!')